In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install torch
!pip install numpy
!pip install pronouncing
!pip install frozendict
!pip install sacremoses
!pip install Phyme

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 32.7 MB/s 
     |████████████████████████████████| 596 kB 69.5 MB/s 
     |████████████████████████████████| 101 kB 14.7 MB/s 
     |████████████████████████████████| 6.6 MB 48.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 939 kB 31.3 MB/s 
  Created wheel for pronouncing: filename=pronouncing-0.2.0-py2.py3-none-any.whl size=6252 sha256=3c43c1ed7051eb625d7852015e7d65f484551b227707d605a7751e0d4b38c79e
  

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
%cd drive/MyDrive/eth_courses/Autumn22/nlp/spanningtrees/

/content/drive/MyDrive/eth_courses/Autumn22/nlp/spanningtrees


In [ ]:
from poem_generation import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


100%|██████████| 267735/267735 [00:00<00:00, 478749.28it/s]


In [ ]:
model = model.to(torch.device('cuda'))
device = torch.device('cuda')

In [ ]:
len(get_rhyming_words("lonely")),
len(get_rhyming_words("peering")),
len(get_rhyming_words("discovered")),

(12298,)

(5470,)

(860,)

In [ ]:
class BanNonWords(LogitsProcessor):
    def __init__(self):
        # We will allow the first three: <eos> , .
        # We tried disallowing <eos> but for some reason this sometimes makes the probability inf?? idk why.
        # Unclear why.
        self.vocab_invalid_idx = [idx for idx, sym in enumerate(vocab['idx2sym']) if not sym in cmu][3:]

    def __call__(self, input_ids, scores):
        banned_tokens = []
        # for every beam (partially generated sentence)
        for beam_index, (beam_input_ids, beam_scores) in enumerate(zip(input_ids, scores)):
            banned_tokens.append(self.vocab_invalid_idx)
            beam_scores[0] -= 5.0
        # set the scores of all banned tokens over the beams to -inf
        scores = set_scores_to_inf_for_banned_tokens(scores, banned_tokens)
        return scores

lcp = LineCommaPolice([8])
poem_processor = LineRhymer(num_syllables=[8], sd=4.2) # LineRhymer(num_syllables=[6], sd = 4.2)
word_banning_processor = BanNonWords()


# logits_processor = LogitsProcessorList([long_word_encourager()])
logits_processor = LogitsProcessorList([lcp, poem_processor, word_banning_processor])

prompt = 'The wind rushed towards the man who lifted the spear from'
prompt = 'Over hill, over dale,'
prompt = 'Standing here all lonely, wrapped in my own misery, I thought upon the only'
prompt='Deep into that darkness peering,'
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids'].to(device)
outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=3,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=50,
    do_sample=True,
    top_p=0.92,
    top_k=0
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [8]))
  print('##########')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


num_syllables: 0
Deep into that darkness peering,
[('Deep', 1), ('into', 2), ('that', 1), ('darkness', 2), ('peering', 2), (',', 0)]
2
num_syllables: 0
Deep into that darkness peering,
[('Deep', 1), ('into', 2), ('that', 1), ('darkness', 2), ('peering', 2), (',', 0)]
2
num_syllables: 0
Deep into that darkness peering,
[('Deep', 1), ('into', 2), ('that', 1), ('darkness', 2), ('peering', 2), (',', 0)]
2
num_syllables: 1
Deep into that darkness peering, the
[('Deep', 1), ('into', 2), ('that', 1), ('darkness', 2), ('peering', 2), (',', 0), ('the', 1)]
3
num_syllables: 2
Deep into that darkness peering, matured
[('Deep', 1), ('into', 2), ('that', 1), ('darkness', 2), ('peering', 2), (',', 0), ('matured', 2)]
4
num_syllables: 0
Deep into that darkness peering,,
[('Deep', 1), ('into', 2), ('that', 1), ('darkness', 2), ('peering', 2), (',', 0), (',', 0)]
2
num_syllables: 3
Deep into that darkness peering, the only
[('Deep', 1), ('into', 2), ('that', 1), ('darkness', 2), ('peering', 2), (',', 0

In [ ]:
lcp = LineCommaPolice([8])
poem_processor = LineRhymer(num_syllables=[8], sd=4.2) # LineRhymer(num_syllables=[6], sd = 4.2)
word_banning_processor = BanNonWords()


# logits_processor = LogitsProcessorList([long_word_encourager()])
logits_processor = LogitsProcessorList([lcp, poem_processor, word_banning_processor])

prompt = 'The wind rushed towards the man who lifted the spear from'
prompt = 'Over hill, over dale,'
prompt = 'Standing here all lonely, wrapped in my own misery, I thought upon the only'
prompt='Deep into that darkness peering,'
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids'].to(device)
outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=3,
    num_beams=8,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=50,
    do_sample=False,
    # top_p=0.92,
    # top_k=0
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [8]))
  print('##########')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


num_syllables: 0
Deep into that darkness peering,
[('Deep', 1), ('into', 2), ('that', 1), ('darkness', 2), ('peering', 2), (',', 0)]
2
num_syllables: 0
Deep into that darkness peering,
[('Deep', 1), ('into', 2), ('that', 1), ('darkness', 2), ('peering', 2), (',', 0)]
2
num_syllables: 0
Deep into that darkness peering,
[('Deep', 1), ('into', 2), ('that', 1), ('darkness', 2), ('peering', 2), (',', 0)]
2
num_syllables: 0
Deep into that darkness peering,
[('Deep', 1), ('into', 2), ('that', 1), ('darkness', 2), ('peering', 2), (',', 0)]
2
num_syllables: 0
Deep into that darkness peering,
[('Deep', 1), ('into', 2), ('that', 1), ('darkness', 2), ('peering', 2), (',', 0)]
2
num_syllables: 0
Deep into that darkness peering,
[('Deep', 1), ('into', 2), ('that', 1), ('darkness', 2), ('peering', 2), (',', 0)]
2
num_syllables: 0
Deep into that darkness peering,
[('Deep', 1), ('into', 2), ('that', 1), ('darkness', 2), ('peering', 2), (',', 0)]
2
num_syllables: 0
Deep into that darkness peering,
[('De

In [ ]:
lcp = LineCommaPolice([6, 8])
poem_processor = LineRhymer(num_syllables=[6, 8], sd=4.2) # LineRhymer(num_syllables=[6], sd = 4.2)
word_banning_processor = BanNonWords()


# logits_processor = LogitsProcessorList([long_word_encourager()])
logits_processor = LogitsProcessorList([lcp, poem_processor, word_banning_processor])

prompt = 'The wind rushed towards the man who lifted the spear from'
prompt = 'Over hill, over dale,'
prompt = 'Standing here all lonely' #, wrapped in my own misery, I thought upon the only'
# prompt='Deep into that darkness peering,'
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids'].to(device)
outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=2,
    num_beams=8,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=40,
    do_sample=False,
    # top_p=0.92,
    # top_k=0
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [8]))
  print('##########')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


num_syllables: 0
Standing here all lonely
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2)]
2
num_syllables: 0
Standing here all lonely
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2)]
2
num_syllables: 0
Standing here all lonely
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2)]
2
num_syllables: 0
Standing here all lonely
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2)]
2
num_syllables: 0
Standing here all lonely
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2)]
2
num_syllables: 0
Standing here all lonely
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2)]
2
num_syllables: 0
Standing here all lonely
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2)]
2
num_syllables: 0
Standing here all lonely
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2)]
2
num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
2
num_syllables: 0
Standing here all lonely.
[('Standing', 2), ('here', 1

In [ ]:
lcp = LineCommaPolice([6])
poem_processor = LineRhymer(num_syllables=[6], sd=4.2) # LineRhymer(num_syllables=[6], sd = 4.2)
word_banning_processor = BanNonWords()


# logits_processor = LogitsProcessorList([long_word_encourager()])
logits_processor = LogitsProcessorList([lcp, poem_processor, word_banning_processor])

prompt = 'The wind rushed towards the man who lifted the spear from'
prompt = 'Over hill, over dale,'
prompt = 'Standing here all lonely, wrapped in my own misery, I thought upon the only'
prompt='Standing here all lonely,'
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids'].to(device)
outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=2,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=30,
    do_sample=True,
    top_p=0.92,
    top_k=0
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6, 6]))
  print('##########')

In [ ]:
lcp = LineCommaPolice([6])
poem_processor = LineRhymer(num_syllables=[6], sd=4.2) # LineRhymer(num_syllables=[6], sd = 4.2)
word_banning_processor = BanNonWords()


# logits_processor = LogitsProcessorList([long_word_encourager()])
logits_processor = LogitsProcessorList([lcp, poem_processor, word_banning_processor])

prompt = 'The wind rushed towards the man who lifted the spear from'
prompt = 'Over hill, over dale,'
prompt = 'Standing here all lonely, wrapped in my own misery, I thought upon the only'
prompt='Standing here all lonely,'
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids'].to(device)
outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=2,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=30,
    do_sample=True,
    top_p=0.92,
    top_k=0
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6, 6]))
  print('##########')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
4
num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
4
num_syllables: 1
Standing here all lonely, and
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('and', 1)]
5
num_syllables: 2
Standing here all lonely, either
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('either', 2)]
6
num_syllables: 0
Standing here all lonely, and originally
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('and', 1), ('originally', 5)]
4
num_syllables: 0
Standing here all lonely, either ostensibly
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('either', 2), ('ostensibly', 4)]
4
num_syllables: 0
Standing here all lonely, and originally,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('and', 1), ('originally', 5), (',', 0)]
4
num_syllables: 0

In [ ]:
lcp = LineCommaPolice([6])
poem_processor = LineRhymer(num_syllables=[6], sd=4.2) # LineRhymer(num_syllables=[6], sd = 4.2)
word_banning_processor = BanNonWords()


# logits_processor = LogitsProcessorList([long_word_encourager()])
logits_processor = LogitsProcessorList([lcp, poem_processor, word_banning_processor])

prompt = 'The wind rushed towards the man who lifted the spear from'
prompt = 'Over hill, over dale,'
prompt = 'Standing here all lonely, wrapped in my own misery, I thought upon the only'
prompt='Standing here all lonely,'
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids'].to(device)
outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=2,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=30,
    do_sample=True,
    top_p=0.92,
    top_k=0
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6, 6]))
  print('##########')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
4
num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
4
num_syllables: 1
Standing here all lonely, or
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('or', 1)]
5
num_syllables: 2
Standing here all lonely, lonely
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('lonely', 2)]
6
num_syllables: 3
Standing here all lonely, or squalor
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('or', 1), ('squalor', 2)]
7
num_syllables: 4
Standing here all lonely, lonely depressed
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('lonely', 2), ('depressed', 2)]
8
num_syllables: 3
Standing here all lonely, or squalor.
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('or', 1), ('squalor', 2), ('.', 0)]
7
num_syllables: 0
Standing here all l

RuntimeError: ignored

In [ ]:
lcp = LineCommaPolice([6])
poem_processor = LineRhymer(num_syllables=[6], sd = 4.2)
word_banning_processor = BanNonWords()


# logits_processor = LogitsProcessorList([long_word_encourager()])
logits_processor = LogitsProcessorList([lcp, poem_processor, word_banning_processor])

prompt = 'The wind rushed towards the man who lifted the spear from'
prompt = 'Over hill, over dale,'
prompt = 'Standing here all lonely, wrapped in my own misery, I thought upon the only'
prompt='Standing here all lonely,'
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids'].to(device)
outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=2,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=30,
    do_sample=True,
    top_p=0.92,
    top_k=0
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6, 6]))
  print('##########')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
4
num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
4
num_syllables: 1
Standing here all lonely, is
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('is', 1)]
5
num_syllables: 2
Standing here all lonely, lonely
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('lonely', 2)]
6
num_syllables: 0
Standing here all lonely, is terminology
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('is', 1), ('terminology', 5)]
4
num_syllables: 0
Standing here all lonely, lonely genuinely
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('lonely', 2), ('genuinely', 4)]
4
num_syllables: 0
Standing here all lonely, is terminology,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('is', 1), ('terminology', 5), (',', 0)]
4
num_syllables: 0
Sta

In [ ]:
lcp = LineCommaPolice([6])
poem_processor = LineRhymer(num_syllables=[6], sd = 4.2)
word_banning_processor = BanNonWords()


# logits_processor = LogitsProcessorList([long_word_encourager()])
logits_processor = LogitsProcessorList([lcp, poem_processor, word_banning_processor])

prompt = 'The wind rushed towards the man who lifted the spear from'
prompt = 'Over hill, over dale,'
prompt = 'Standing here all lonely, wrapped in my own misery, I thought upon the only'
prompt='Standing here all lonely,'
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids'].to(device)
outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=2,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=30,
    do_sample=True,
    top_p=0.92,
    top_k=0
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6, 6]))
  print('##########')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
4
num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
4
num_syllables: 1
Standing here all lonely, next
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('next', 1)]
5
num_syllables: 2
Standing here all lonely, within
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('within', 2)]
6
num_syllables: 2
Standing here all lonely, next all
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('next', 1), ('all', 1)]
6
num_syllables: 0
Standing here all lonely, within progressively
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('within', 2), ('progressively', 4)]
4
num_syllables: 0
Standing here all lonely, next all physically
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('next', 1), ('all', 1), ('physically', 4)]
4
num_syllables

In [ ]:
vocab['idx2sym'][:5]

['<eos>', 'the', ',', '.', 'of']

In [ ]:
lcp = LineCommaPolice([6, 9, 4])
poem_processor = LineRhymer(num_syllables=[6, 9, 4], sd = 4.2)
word_banning_processor = BanNonWords()


# logits_processor = LogitsProcessorList([long_word_encourager()])
logits_processor = LogitsProcessorList([lcp, poem_processor, word_banning_processor])

prompt = 'The wind rushed towards the man who lifted the spear from'
prompt = 'Over hill, over dale,'
# prompt = 'Standing here all lonely, wrapped in my own misery, I thought upon the only'
# prompt='Standing here all lonely,'
prompt = "For what are you searching,"
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids'].to(device)
outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=2,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=30,
    do_sample=True,
    top_p=0.92,
    top_k=0
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6, 9, 4]))
  print('##########')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


num_syllables: 0
For what are you searching,
[('For', 1), ('what', 1), ('are', 1), ('you', 1), ('searching', 2), (',', 0)]
1
num_syllables: 0
For what are you searching,
[('For', 1), ('what', 1), ('are', 1), ('you', 1), ('searching', 2), (',', 0)]
1
num_syllables: 1
For what are you searching, and
[('For', 1), ('what', 1), ('are', 1), ('you', 1), ('searching', 2), (',', 0), ('and', 1)]
2
num_syllables: 1
For what are you searching, and
[('For', 1), ('what', 1), ('are', 1), ('you', 1), ('searching', 2), (',', 0), ('and', 1)]
2
num_syllables: 2
For what are you searching, and a
[('For', 1), ('what', 1), ('are', 1), ('you', 1), ('searching', 2), (',', 0), ('and', 1), ('a', 1)]
3
num_syllables: 3
For what are you searching, and entered
[('For', 1), ('what', 1), ('are', 1), ('you', 1), ('searching', 2), (',', 0), ('and', 1), ('entered', 2)]
4
num_syllables: 5
For what are you searching, and a selected
[('For', 1), ('what', 1), ('are', 1), ('you', 1), ('searching', 2), (',', 0), ('and', 1), 

In [ ]:
float("inf")/float("inf")

nan

In [ ]:
for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6, 9, 4]))
  print('##########')

 Over hill , over dale ,
just to the north .
##########
 Over hill , over dale ,
dale and a couple of , eight and pale
just six months , tail
chestnut , elm and pine , pale
barns , and
##########


In [ ]:
len(get_rhyming_words("dale")),
len(get_rhyming_words("lonely")),
len(get_rhyming_words("sky")),
len(get_rhyming_words("man")),
len(get_rhyming_words("ending"))

(153,)

(12298,)

(201,)

(198,)

5470

In [ ]:
from poem_generation import *

In [ ]:
class BanNonWords(LogitsProcessor):
    def __init__(self):
        # We will allow the first three: <eos> , . Actually won't allow <eos> as
        # my poems get too short
        self.vocab_invalid_idx = [0] + [idx for idx, sym in enumerate(vocab['idx2sym']) if not sym in cmu][3:]

    def __call__(self, input_ids, scores):
        banned_tokens = []
        # for every beam (partially generated sentence)
        for beam_index, (beam_input_ids, beam_scores) in enumerate(zip(input_ids, scores)):
            banned_tokens.append(self.vocab_invalid_idx)
        # set the scores of all banned tokens over the beams to -inf
        scores = set_scores_to_inf_for_banned_tokens(scores, banned_tokens)
        return scores

lcp = LineCommaPolice([6])
poem_processor = LineRhymer(num_syllables=[6, 9, 4], sd = 4.2)
word_banning_processor = BanNonWords()


# logits_processor = LogitsProcessorList([long_word_encourager()])
logits_processor = LogitsProcessorList([lcp, poem_processor, word_banning_processor])

prompt = 'The wind rushed towards the man who lifted the spear from'
prompt = 'For what are you searching?,'
# prompt = 'Standing here all lonely, wrapped in my own misery, I thought upon the only'
# prompt='Standing here all lonely,'
prompt = ""
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids'].to(device)
outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=2,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=30,
    do_sample=True,
    top_p=0.92,
    top_k=0
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6, 9, 4]))
  print('##########')